<font size="4">**Детализированная картина. Расчет плотности финансовых организаций (ФО) по регионам РФ и построение тепловой карты с учетом метрик плотности:**</font>

<font size="4">- количество организаций на квадратный километр (density_per_km2);</font>

<font size="4">- количество организаций на 100 тысяч населения (density_per_100k).</font>

<font size="4">**Часть 2. Подготовка данных для построения дашборда в Yandex Datalens.**</font>

In [202]:
import pandas as pd
import numpy as np
import re

# Загрузка данных
df = pd.read_excel('Свод ГО и подразделения.xlsx') 
regions_df = pd.read_excel('russia_data.xlsx')

In [204]:
# Словарь для названий полигонов (сохраняем скобки)
polygon_names = {
    'Белгородская обл': 'Белгородская',
    'Брянская обл': 'Брянская',
    'Владимирская обл': 'Владимирская',
    'Воронежская обл': 'Воронежская',
    'Ивановская обл': 'Ивановская',
    'Калужская обл': 'Калужская',
    'Костромская обл': 'Костромская',
    'Курская обл': 'Курская',
    'Липецкая обл': 'Липецкая',
    'Московская обл': 'Московская',
    'Орловская обл': 'Орловская',
    'Рязанская обл': 'Рязанская',
    'Смоленская обл': 'Смоленская',
    'Тамбовская обл': 'Тамбовская',
    'Тверская обл': 'Тверская',
    'Тульская обл': 'Тульская',
    'Ярославская обл': 'Ярославская',
    'г Москва': 'Москва',
    'Респ Карелия': 'Карелия',
    'Респ Коми': 'Коми',
    'Архангельская обл': 'Архангельская',
    'Вологодская обл': 'Вологодская',
    'Калининградская обл': 'Калининградская',
    'Ленинградская обл': 'Ленинградская',
    'Мурманская обл': 'Мурманская',
    'Новгородская обл': 'Новгородская',
    'Псковская обл': 'Псковская',
    'г Санкт-Петербург': 'Санкт-Петербург',
    'Ненецкий АО': 'Ненецкий',
    'Респ Адыгея': 'Адыгея',
    'Респ Калмыкия': 'Калмыкия',
    'Респ Крым': 'Крым',
    'Краснодарский край': 'Краснодарский',
    'Астраханская обл': 'Астраханская',
    'Волгоградская обл': 'Волгоградская',
    'Ростовская обл': 'Ростовская',
    'г Севастополь': 'Севастополь',
    'Респ Дагестан': 'Дагестан',
    'Респ Ингушетия': 'Ингушетия',
    'Кабардино-Балкарская Респ': 'Кабардино-Балкария',
    'Карачаево-Черкесская Респ': 'Карачаево-Черкесия',
    'Респ Северная Осетия - Алания': 'Северная Осетия - Алания',
    'Чеченская Респ': 'Чечня',
    'Ставропольский край': 'Ставропольский',
    'Респ Башкортостан': 'Башкортостан',
    'Респ Марий Эл': 'Марий Эл',
    'Респ Мордовия': 'Мордовия',
    'Респ Татарстан': 'Татарстан',
    'Удмуртская Респ': 'Удмуртия',
    'Чувашская Республика - Чувашия': 'Чувашия',
    'Пермский край': 'Пермский',
    'Кировская обл': 'Кировская',
    'Нижегородская обл': 'Нижегородская',
    'Оренбургская обл': 'Оренбургская',
    'Пензенская обл': 'Пензенская',
    'Самарская обл': 'Самарская',
    'Саратовская обл': 'Саратовская',
    'Ульяновская обл': 'Ульяновская',
    'Курганская обл': 'Курганская',
    'Свердловская обл': 'Свердловская',
    'Тюменская обл': 'Тюменская',
    'Челябинская обл': 'Челябинская',
    'Ханты-Мансийский Автономный округ - Югра': 'Ханты-Мансийский (Югра)',
    'Ямало-Ненецкий АО': 'Ямало-Ненецкий',
    'Респ Алтай': 'Алтай',
    'Респ Тыва': 'Тыва',
    'Респ Хакасия': 'Хакасия',
    'Алтайский край': 'Алтайский',
    'Красноярский край': 'Красноярский',
    'Иркутская обл': 'Иркутская',
    'Кемеровская область - Кузбасс': 'Кемеровская',
    'Новосибирская обл': 'Новосибирская',
    'Омская обл': 'Омская',
    'Томская обл': 'Томская',
    'Респ Бурятия': 'Бурятия',
    'Респ Саха Якутия': 'Саха (Якутия)',
    'Забайкальский край': 'Забайкальский',
    'Камчатский край': 'Камчатский',
    'Приморский край': 'Приморский',
    'Хабаровский край': 'Хабаровский',
    'Амурская обл': 'Амурская',
    'Магаданская обл': 'Магаданская',
    'Сахалинская обл': 'Сахалинская',
    'Еврейская Аобл': 'Еврейская',
    'Чукотский АО': 'Чукотский',
    'Луганская Народная респ': 'Луганская',
    'Донецкая Народная респ': 'Донецкая',
    'Запорожская обл': 'Запорожская',
    'Херсонская обл': 'Херсонская'
}

In [206]:
# Функция для очистки специальных символов (сохраняем скобки для названий полигонов)
def clean_special_characters(text, preserve_brackets=False):
    """Удаляем специальные символы из текста"""
    if pd.isna(text):
        return text
    
    if preserve_brackets:
        # Сохраняем скобки
        return re.sub(r'[^\w\s\-\.\(\)]', '', str(text))
    else:
        # Удаляем все, включая скобки
        return re.sub(r'[^\w\s\-\.]', '', str(text))

In [208]:
# Очищаем текстовые колонки (без сохранения скобок)
text_columns = ['federal_district', 'region', 'tp']
for col in text_columns:
    if col in regions_df.columns:
        regions_df[col] = regions_df[col].apply(lambda x: clean_special_characters(x, False))
    if col in df.columns:
        df[col] = df[col].apply(lambda x: clean_special_characters(x, False))

In [210]:
# Добавляем название полигона (сохраняем скобки)
regions_df['Название полигона'] = regions_df['region'].map(polygon_names)

# Получаем все уникальные типы организаций
org_types = df['tp'].unique().tolist()
org_types = [t for t in org_types if pd.notna(t)]

In [212]:
# Создаем базовый датафрейм с регионами и их данными
base_columns = ['federal_district', 'region', 'Название полигона', 'area_km2', 'population']
result_rows = []

# Обрабатываем числовые колонки
regions_df['area_km2'] = regions_df['area_km2'].astype(str).str.replace(' ', '').astype(float)
regions_df['population'] = regions_df['population'].astype(str).str.replace(' ', '').astype(float)

In [214]:
# Для каждого региона создаем записи для всех видов организаций
for _, region_row in regions_df.iterrows():
    region_name = region_row['region']
    
    # Получаем данные по организациям в регионе
    region_orgs = df[df['region'] == region_name]
    total_orgs = len(region_orgs)
    
    # Создаем запись для "Все организации"
    all_orgs_row = {
        'federal_district': region_row['federal_district'],
        'region': region_name,
        'Название полигона': region_row['Название полигона'],  # Здесь сохраняются скобки
        'Вид организации': 'Все организации',
        'area_km2': region_row['area_km2'],
        'population': region_row['population'],
        'Количество организаций': total_orgs
    }
    result_rows.append(all_orgs_row)
    
    # Создаем записи для каждого типа организаций
    for org_type in org_types:
        org_count = len(region_orgs[region_orgs['tp'] == org_type])
        
        org_type_row = {
            'federal_district': region_row['federal_district'],
            'region': region_name,
            'Название полигона': region_row['Название полигона'],  # Здесь сохраняются скобки
            'Вид организации': org_type,
            'area_km2': region_row['area_km2'],
            'population': region_row['population'],
            'Количество организаций': org_count
        }
        result_rows.append(org_type_row)

In [216]:
# Создаем итоговый датафрейм
result_df = pd.DataFrame(result_rows)

# Рассчитываем плотности
result_df['density_per_1000km2'] = result_df['Количество организаций'] / result_df['area_km2'] * 1000
result_df['density_per_100k'] = (result_df['Количество организаций'] / result_df['population']) * 100000

In [218]:
# Заменяем бесконечные значения на 0
result_df = result_df.replace([np.inf, -np.inf], 0)

In [220]:
# Функция для полной очистки данных (сохраняем скобки в названиях полигонов)
def thoroughly_clean_data(df):
    """Тщательно очищает данные для Яндекс Даталенс"""
    
    # Очищаем все текстовые колонки, кроме названия полигона
    for col in df.columns:
        if df[col].dtype == 'object':
            if col == 'Название полигона':
                # Для названия полигона сохраняем скобки
                df[col] = df[col].apply(lambda x: clean_special_characters(x, True))
            else:
                # Для остальных колонок удаляем все специальные символы
                df[col] = df[col].apply(lambda x: clean_special_characters(x, False))
    
    # Преобразуем числовые колонки
    numeric_cols = ['area_km2', 'population', 'Количество организаций', 
                   'density_per_1000km2', 'density_per_100k']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    # Очищаем названия столбцов
    df.columns = [re.sub(r'[^\w]', '_', str(col)) for col in df.columns]
    df.columns = [col.strip('_') for col in df.columns]
    
    # Заполняем пропуски
    df = df.fillna(0)
    
    return df

In [222]:
# Применяем очистку
cleaned_df = thoroughly_clean_data(result_df)

In [224]:
# Проверяем результат
print("Проверка очищенных данных:")
print(f"Размер данных: {cleaned_df.shape}")
print(f"Типы данных:\n{cleaned_df.dtypes}")
print(f"Уникальные виды организаций: {cleaned_df['Вид_организации'].unique()}")
print(f"Пример названий полигонов: {cleaned_df['Название_полигона'].unique()[:10]}")

Проверка очищенных данных:
Размер данных: (5340, 9)
Типы данных:
federal_district           object
region                     object
Название_полигона          object
Вид_организации            object
area_km2                  float64
population                float64
Количество_организаций      int64
density_per_1000km2       float64
density_per_100k          float64
dtype: object
Уникальные виды организаций: ['Все организации' 'Микрокредитные компании' 'Ломбарды'
 'Кредитные потребительские кооперативы'
 'Сельскохозяйственные кредитные потребительские кооперативы'
 'Операторы по приему платежей' 'Кредитные организации' 'Брокеры' 'Дилеры'
 'Специализированные депозитарии ИФ ПИФ и НПФ' 'Регистраторы'
 'Управляющие компании ИФ ПИФ и НПФ' 'Депозитарии'
 'Саморегулируемые организации в сфере финансового рынка'
 'СРО Депозитариев' 'СРО Управляющих'
 'СРО Специализированных депозитариев'
 'Управляющие компании специализированных обществ' 'Страховые брокеры'
 'Микрофинансовые компании' 'Инве

In [226]:
# Сохраняем в разных форматах
cleaned_df.to_csv('data_for_datalens.csv', index=False, encoding='utf-8-sig')
cleaned_df.to_excel('cleaned_data_for_datalens.xlsx', index=False, engine='openpyxl')

print("\nДанные успешно подготовлены для Яндекс Даталенс!")


Данные успешно подготовлены для Яндекс Даталенс!


<font size="4">Подготовка файла с полигонами для создания карты</font>

In [153]:
def parse_polygon_data(line):
    """Парсит строку с учетом структуры полигонов в тройных скобках"""
    # Ищем паттерн: начало полигона с [[[ и конец с ]]]
    polygon_match = re.search(r'(\[\[\[.*?\]\]\])', line)
    
    if polygon_match:
        polygon = polygon_match.group(1)
        # Разделяем строку до полигона
        before_polygon = line[:polygon_match.start()].rstrip()
        # Разделяем оставшуюся часть на поля
        fields = before_polygon.split('\t')
        
        # Убедимся, что у нас 5 полей до полигона
        if len(fields) >= 5:
            return fields[:5] + [polygon]
        else:
            # Если меньше 5 полей, заполняем недостающие
            return fields + [''] * (5 - len(fields)) + [polygon]
    else:
        # Если полигон не найден, просто разделяем по табам
        fields = line.split('\t')
        return fields + [''] * (6 - len(fields))

In [155]:
# Читаем файл построчно
data = []
with open('polygons_data.csv', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f):
        line = line.strip()
        if line and not line.startswith('Номер'):  # Пропускаем заголовок и пустые строки
            parsed = parse_polygon_data(line)
            data.append(parsed)

print(f"Прочитано строк данных: {len(data)}")

Прочитано строк данных: 85


In [157]:
# Создаем DataFrame
df = pd.DataFrame(data, columns=[
    'Номер', 'Полигон_ID', 'Регион', 'Тип_полигона', 'Название_полигона', 'Полигон'
])

print(f"Создан DataFrame: {df.shape}")

Создан DataFrame: (85, 6)


In [161]:
df.head(3)

,Номер,Полигон_ID,Регион,Тип_полигона,Название_полигона,Полигон
0,1,29,Российская Федерация,Регион,Московская,"[[[55.449809499989271,37.49815720008877],[55.4..."
1,2,64,Российская Федерация,Регион,Ростовская,"[[[50.2144768997274,41.350254599602977],[50.20..."
2,3,65,Российская Федерация,Регион,Рязанская,"[[[55.364243999940754,40.210444700223718],[55...."


In [163]:
# Сохраняем с правильным форматированием
df.to_csv('polygons_processed.csv', index=False, encoding='utf-8-sig')

print("Данные успешно обработаны и сохранены в polygons_processed.csv")

Данные успешно обработаны и сохранены в polygons_processed.csv
